In [1]:
# third party

# syft absolute
import syft as sy

sy.__version__

'0.8.5-post.1'

In [2]:
try:
    gw_node.land()
    dm_node1.land()
    dm_node2.land()
except Exception:
    pass

In [3]:
gw_node = sy.orchestra.launch(
    name="test-gateway-1", dev_mode=True, reset=True, node_type="gateway"
)
dm_node1 = sy.orchestra.launch(
    name="test-domain-1", dev_mode=True, reset=True, node_type="domain"
)
dm_node2 = sy.orchestra.launch(
    name="test-domain-2", dev_mode=True, reset=True, node_type="domain"
)

gw_client = gw_node.login(email="info@openmined.org", password="changethis")
dm_client1 = dm_node1.login(email="info@openmined.org", password="changethis")
dm_client2 = dm_node2.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=da023a5e9fae4203b78e04246f686e03 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=622acafa70fe4dd69f10787b127ab6e4 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=d54d01a482da44a6a9577995d50df7ec in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <test-gateway-1: High side Gateway> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test-domain-2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
dm_client1.connect_to_gateway(gw_client)
dm_client2.connect_to_gateway(gw_client)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=622acafa70fe4dd69f10787b127ab6e4 in_memory=True
Default worker pool error. Invalid number of workers: 0
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=da023a5e9fae4203b78e04246f686e03 in_memory=True
Default worker pool error. Invalid number of workers: 0
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=622acafa70fe4dd69f10787b127ab6e4 in_memory=True
Default worker pool error. Invalid number of workers: 0
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=da023a5e9fae4203b78e04246f686e03 in_memory=True


SyftSuccess: Connected domain test-domain-2 to test-gateway-1 gateway

In [5]:
assert len(gw_client.peers) == 2
gw_client.peers

syft.client.gateway_client.ProxyClient

In [6]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute]

In [7]:
gw_peers[1].node_routes

[syft.service.network.routes.PythonNodeRoute]

#### `add_route_for`: Adding an existed route should update its priority

In [8]:
dm1_peers = dm_client1.api.services.network.get_all_peers()
dm1_peers[0].node_routes[0].priority

1

In [9]:
# stdlib
from copy import deepcopy

# syft absolute
from syft.types.uid import UID

existed_route = deepcopy(dm1_peers[0].node_routes[0])
existed_route.id = UID()

In [10]:
# print(type(gw_client.credentials), gw_client.credentials)
# print(f"{gw_client.metadata = }\n")

# print(type(dm_client1.credentials), dm_client1.credentials)
# print(f"{dm_client1.metadata = }\n")

# print(type(dm_client2.credentials), dm_client2.credentials)
# print(f"{dm_client2.metadata = }\n")

In [11]:
gw_client.api.services.network.add_route_for(peer=gw_peers[0], route=existed_route)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=622acafa70fe4dd69f10787b127ab6e4 in_memory=True
Default worker pool error. Invalid number of workers: 0


SyftSuccess: The route already exists between 'test-domain-1' and peer 'test-gateway-1' with id '01c5464e9af74773b85b7ed2b37cdfc7', so its priority was updated

In [12]:
dm1_peers = dm_client1.api.services.network.get_all_peers()
assert dm1_peers[0].node_routes[0].priority == 2
dm1_peers[0].node_routes[0].priority

2

#### `add_route_for`: Adding a new route for a peer

In [13]:
# syft absolute
from syft.node.worker_settings import WorkerSettings
from syft.service.network.routes import PythonNodeRoute

worker_settings = WorkerSettings.from_node(node=gw_node.python_node)
worker_settings.node_side_type = "low"
new_route = PythonNodeRoute(worker_settings=worker_settings)

new_route

```python
class PythonNodeRoute:
  id: str = b572dc891a144926bbc773ae791246f6

```

In [14]:
gw_client.api.services.network.add_route_for(peer=gw_peers[0], route=new_route)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=622acafa70fe4dd69f10787b127ab6e4 in_memory=True
Default worker pool error. Invalid number of workers: 0


SyftSuccess: New route (PythonNodeRoute) with id 'b572dc891a144926bbc773ae791246f6' to peer 'test-gateway-1' was added for 'test-domain-1'

In [15]:
dm1_peers = dm_client1.api.services.network.get_all_peers()
assert len(dm1_peers[0].node_routes) == 2
dm1_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute,
 syft.service.network.routes.PythonNodeRoute]

In [16]:
for route in dm1_peers[0].node_routes:
    print(f"route {route.id} has priority = {route.priority}")

route 01c5464e9af74773b85b7ed2b37cdfc7 has priority = 2
route b572dc891a144926bbc773ae791246f6 has priority = 3


#### `add_route_for`: Adding another route for a peer

In [17]:
# print(dict(new_route))
# from pprint import pprint
# pprint(dict(worker_settings))

In [18]:
worker_settings2 = WorkerSettings.from_node(node=gw_node.python_node)
worker_settings2.node_side_type = "low"
worker_settings2.id = UID()
worker_settings2

```python
class WorkerSettings:
  id: str = c6ff98fd58ac4b10b2956e87e2a9b456

```

In [19]:
new_route2 = PythonNodeRoute(worker_settings=worker_settings2)
new_route2

```python
class PythonNodeRoute:
  id: str = b5c2edc683034ddebd0eb958d277f8b3

```

In [20]:
gw_client.api.services.network.add_route_for(peer=gw_peers[0], route=new_route2)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=622acafa70fe4dd69f10787b127ab6e4 in_memory=True
Default worker pool error. Invalid number of workers: 0


SyftSuccess: New route (PythonNodeRoute) with id 'b5c2edc683034ddebd0eb958d277f8b3' to peer 'test-gateway-1' was added for 'test-domain-1'

In [21]:
dm1_peers = dm_client1.api.services.network.get_all_peers()
assert len(dm1_peers[0].node_routes) == 3
dm1_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute,
 syft.service.network.routes.PythonNodeRoute,
 syft.service.network.routes.PythonNodeRoute]

In [22]:
for route in dm1_peers[0].node_routes:
    print(f"route {route.id} has priority = {route.priority}")

route 01c5464e9af74773b85b7ed2b37cdfc7 has priority = 2
route b572dc891a144926bbc773ae791246f6 has priority = 3
route b5c2edc683034ddebd0eb958d277f8b3 has priority = 4


#### `delete_route_for`: Delete routes for a peer
Delete using route's id

In [23]:
gw_client.api.services.network.delete_route_for(peer=gw_peers[0], route_id=new_route.id)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=622acafa70fe4dd69f10787b127ab6e4 in_memory=True
Default worker pool error. Invalid number of workers: 0


SyftSuccess: Route with id 'b572dc891a144926bbc773ae791246f6' to peer 'test-gateway-1' was deleted for 'test-domain-1'.

In [24]:
dm1_peers = dm_client1.api.services.network.get_all_peers()
dm1_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute,
 syft.service.network.routes.PythonNodeRoute]

Delete another route using the `NodeRoute` object

In [25]:
dm1_peers[0].node_routes[0]

```python
class PythonNodeRoute:
  id: str = 01c5464e9af74773b85b7ed2b37cdfc7

```

In [26]:
gw_client.api.services.network.delete_route_for(
    peer=gw_peers[0], route=dm1_peers[0].node_routes[0]
)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=622acafa70fe4dd69f10787b127ab6e4 in_memory=True
Default worker pool error. Invalid number of workers: 0


SyftSuccess: Route 'PythonNodeRoute' with id '01c5464e9af74773b85b7ed2b37cdfc7' to peer 'test-gateway-1' was deleted for 'test-domain-1'.

In [27]:
dm1_peers = dm_client1.api.services.network.get_all_peers()
dm1_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute]

delete the last route for the domain1 peer

In [28]:
gw_client.api.services.network.delete_route_for(
    peer=gw_peers[0], route=dm1_peers[0].node_routes[0]
)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=622acafa70fe4dd69f10787b127ab6e4 in_memory=True
Default worker pool error. Invalid number of workers: 0


SyftWarning: There is only one route left to peer 'test-gateway-1'. Removing this route will remove the peer for 'test-domain-1'.

Would you like to proceed? [y/n]:  y


SyftSuccess: Route 'PythonNodeRoute' with id 'b5c2edc683034ddebd0eb958d277f8b3' to peer 'test-gateway-1' was deleted for 'test-domain-1'. There is no routes left to connect to peer 'test-gateway-1', so it is deleted for 'test-domain-1'.

In [29]:
dm1_peers = dm_client1.api.services.network.get_all_peers()
dm1_peers

[]

In [30]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers

[syft.service.network.node_peer.NodePeer,
 syft.service.network.node_peer.NodePeer]

#### `delete_route` and `add_route`: Test if `add_route` and `delete_route` can be called on self to delete a route for one of its peers

In [31]:
gw_client.api.services.network.delete_route(peer=gw_peers[0], route=gw_peers[0].node_routes[0])

SyftWarning: There is only one route left to peer 'test-domain-1'. Removing this route will remove the peer for 'test-gateway-1'.

Would you like to proceed? [y/n]:  y


SyftSuccess: Route 'PythonNodeRoute' with id '2c50dba106134f5f84995c684fb91338' to peer 'test-domain-1' was deleted for 'test-gateway-1'. There is no routes left to connect to peer 'test-domain-1', so it is deleted for 'test-gateway-1'.

In [32]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers

[syft.service.network.node_peer.NodePeer]

In [33]:
gw_client.api.services.network.add_route(peer=gw_peers[0], route=new_route)

SyftSuccess: New route (PythonNodeRoute) with id 'b572dc891a144926bbc773ae791246f6' to peer 'test-domain-2' was added for 'test-gateway-1'

In [34]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute,
 syft.service.network.routes.PythonNodeRoute]

#### Update a route's priority

In [24]:
# syft absolute
from syft.node.worker_settings import WorkerSettings
from syft.service.network.routes import PythonNodeRoute

worker_settings = WorkerSettings.from_node(node=gw_node.python_node)
worker_settings.node_side_type = "low"
new_route = PythonNodeRoute(worker_settings=worker_settings)

new_route

```python
class PythonNodeRoute:
  id: str = 8ec6da1e558644e79b422350afee496a

```

In [25]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute]

In [26]:
gw_client.api.services.network.add_route_for(peer=gw_peers[0], route=new_route)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=2f73d519be654bd393f3da5ff1e610c7 in_memory=True
Default worker pool error. Invalid number of workers: 0


SyftSuccess: New route with id '8ec6da1e558644e79b422350afee496a' added for test-domain-2

In [27]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute,
 syft.service.network.routes.PythonNodeRoute]

Not specifying new priority: it will be automatically assigned to has the highest priority

In [28]:
gw_client.api.services.network.update_route_priority_for(
    peer=gw_peers[0], route=new_route
)

SyftSuccess: Route 8ec6da1e558644e79b422350afee496a's priority updated to 3 for peer test-domain-2

In [29]:
gw_peers = gw_client.api.services.network.get_all_peers()
for route in gw_peers[0].node_routes:
    print(f"route {route.id} has priority = {route.priority}")

route 9ad2b515288b4f919af5bc8ef563a0ef has priority = 1
route 8ec6da1e558644e79b422350afee496a has priority = 3


When specifying new priority

In [30]:
gw_client.api.services.network.update_route_priority_for(
    peer=gw_peers[0], route=new_route, priority=5
)

SyftSuccess: Route 8ec6da1e558644e79b422350afee496a's priority updated to 5 for peer test-domain-2

In [31]:
gw_peers = gw_client.api.services.network.get_all_peers()
for route in gw_peers[0].node_routes:
    print(f"route {route.id} has priority = {route.priority}")

route 9ad2b515288b4f919af5bc8ef563a0ef has priority = 1
route 8ec6da1e558644e79b422350afee496a has priority = 5


### Launch the first domain node 1 again but with different port

In [ ]:
dm_node1 = sy.orchestra.launch(
    name="test-domain-1", port=9098, dev_mode=True, reset=True, node_type="domain"
)
dm_client1 = dm_node1.login(
    email="info@openmined.org", password="changethis", port=9098
)

In [ ]:
dm_client1.connect_to_gateway(gw_client)

In [ ]:
dm_client1.peers[0].node_routes

In [ ]:
gw_client.peers

In [ ]:
dm_client1.peers